In [6]:
from imports import importData
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF
import time
import numpy as np
import pandas as pd

def getGPData(combCD4=True):
    #import data
    fullData = importData()

    HotEnc = OneHotEncoder()
    HotEnc.fit(fullData[['Cell']])
    HotEncName = HotEnc.transform(fullData[['Cell']]).toarray()
    HotEncName = pd.DataFrame(HotEncName)

    xData = fullData[["Dose","Time","IL2RaKD","IL2RBGKD","IL15Ra","IL2Ra","IL2Rb","IL7Ra","gc","Bivalent"]]
    xData[['Dose', 'IL2RaKD', 'IL2RBGKD']] = np.log10(xData[['Dose', 'IL2RaKD', 'IL2RBGKD']])

    yData = fullData[['Mean']]

    cellNames = ['CD8','NK','Thelper','Treg'] 
    for i, name in enumerate(cellNames): #Adds columns of each cell type w/ hot encoding values
        xData[name]=HotEncName.iloc[:,i]

    if combCD4 == True:
        for i in range(len(xData[['Treg']])):  #For every value in Treg Column
            if xData.at[i,'Treg'] == 1: #If it is equal to 1
                xData.at[i,'Thelper'] = 1  #Set Thelper equal to 1 at this index
        
        xData = xData.rename(columns={'Thelper':'CD4'})
        xData = xData.drop(columns=["Treg"])
    
    return xData, yData, fullData

In [7]:
xData,yData,fData = getGPData()

In [8]:
def gaussianProcess(xData, yData, fData, kernType):

    kern_scale = np.ones(len(xData[0]))
    print(kern_scale)

    if kernType == "matern":
        kernel = 1.0 * Matern(length_scale=kern_scale, nu=1.5) 
    elif kernType == "RBF":
        kernel = 1.0 * RBF(1.0)
    elif kernType == "combo*":
        kernel = 1.0 * Matern(length_scale=kern_scale, nu=1.5) * 1.0 * RBF(1.0)
    elif kernType == "combo+":
        kernel = 1.0 * Matern(length_scale=kern_scale, nu=1.5) + 1.0 * RBF(1.0)

    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    #use train_test_split to take random selection
    X_train, X_test, y_train, y_test = train_test_split(xData, yData, test_size=0.25, random_state=0)

    gp.fit(X_train,y_train)
    return gp

In [9]:
def gaussianTest(xData,yData,testSize, kernType):
    
    print(len(xData.columns))
    kern_scale = np.ones(len(xData.columns))
    print(kern_scale)

    if kernType == "matern":
        kernel = 1.0 * Matern(length_scale=kern_scale, nu=1.5) 
    elif kernType == "RBF":
        kernel = 1.0 * RBF(1.0)
    elif kernType == "combo*":
        kernel = 1.0 * Matern(length_scale=kern_scale, nu=1.5) * 1.0 * RBF(1.0)
    elif kernType == "combo+":
        kernel = 1.0 * Matern(length_scale=kern_scale, nu=1.5) + 1.0 * RBF(1.0)

    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    #use train_test_split to take random selection
    X_train, X_test, y_train, y_test = train_test_split(xData, yData, test_size=testSize, random_state=0)
    
    start_time = time.time()
    gp.fit(X_train,y_train)
    end_time = (time.time() - start_time)
    print(str(kernType))
    print("Time: " + str(end_time))

    print("R^2: " + str(gp.score(X_test, y_test)))
    return

SyntaxError: unmatched ')' (<ipython-input-9-d967a591a54d>, line 4)

In [5]:
gaussianTest(xData,yData,0.50, "matern")
gaussianTest(xData,yData,0.50, "RBF")
gaussianTest(xData,yData,0.50, "combo*")
gaussianTest(xData,yData,0.50, "combo+")


KeyError: 0